# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [2]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.2609277823605607
 1.749164741924147
 2.039810543393324

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [3]:
Alu = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.639317  1.0       0.0
 0.166356  0.217595  1.0
U factor:
3×3 Matrix{Float64}:
 0.875702  0.586243   0.577866
 0.0       0.58378   -0.138701
 0.0       0.0        0.824747

In [4]:
typeof(Alu)

LU{Float64, Matrix{Float64}}

The different parts of the factorization can be extracted by accessing their special properties

In [5]:
Alu.P

3×3 Matrix{Float64}:
 0.0  0.0  1.0
 0.0  1.0  0.0
 1.0  0.0  0.0

In [6]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.639317  1.0       0.0
 0.166356  0.217595  1.0

In [7]:
Alu.U

3×3 Matrix{Float64}:
 0.875702  0.586243   0.577866
 0.0       0.58378   -0.138701
 0.0       0.0        0.824747

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [8]:
A\b

3-element Vector{Float64}:
 0.9999999999999999
 1.0
 1.0

In [9]:
Alu\b

3-element Vector{Float64}:
 0.9999999999999999
 1.0
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [10]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [11]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.138803  -0.163146  -0.976789
 -0.533431  -0.818704   0.212544
 -0.834377   0.550552   0.0266113
R factor:
3×3 Matrix{Float64}:
 -1.04953  -1.03165   -0.728873
  0.0      -0.498667  -0.0160755
  0.0       0.0       -0.805604

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [12]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.138803  -0.163146  -0.976789
 -0.533431  -0.818704   0.212544
 -0.834377   0.550552   0.0266113

In [13]:
Aqr.R

3×3 Matrix{Float64}:
 -1.04953  -1.03165   -0.728873
  0.0      -0.498667  -0.0160755
  0.0       0.0       -0.805604

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [14]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -1.0997164991525894
  1.063842629767675
  3.400111823213254
vectors:
3×3 Matrix{Float64}:
 -0.79395    0.341373  -0.503098
  0.042184  -0.794562  -0.605716
  0.606518   0.502131  -0.616442

The values and the vectors can be extracted from the Eigen type by special indexing

In [15]:
AsymEig.values

3-element Vector{Float64}:
 -1.0997164991525894
  1.063842629767675
  3.400111823213254

In [16]:
AsymEig.vectors

3×3 Matrix{Float64}:
 -0.79395    0.341373  -0.503098
  0.042184  -0.794562  -0.605716
  0.606518   0.502131  -0.616442

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [17]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0           1.66533e-15  -3.66374e-15
 -1.11022e-15   1.0           1.77636e-15
 -6.17562e-16  -2.04697e-16   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [18]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [19]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [20]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-2.957651367616715

In [21]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [22]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [23]:
@time eigvals(Asym);

  0.504051 seconds (161.00 k allocations: 16.157 MiB, 44.78% compilation time)


In [24]:
@time eigvals(Asym_noisy);

  1.310442 seconds (13 allocations: 7.920 MiB)


In [25]:
@time eigvals(Asym_explicit);

  0.309379 seconds (5.89 k allocations: 8.316 MiB, 3.57% gc time, 2.75% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [26]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.890319 seconds (803.75 k allocations: 224.542 MiB, 3.65% gc time, 34.77% compilation time)


6.585308499917643

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [27]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [28]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 1//5  2//5   1//5
 1//2  1//1   4//5
 3//5  3//10  3//10

In [29]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
  4//5
 23//10
  6//5

In [30]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [31]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1  0//1  0//1
 5//6  1//1  0//1
 1//3  2//5  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 3//5  3//10   3//10
 0//1  3//4   11//20
 0//1  0//1   -3//25

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [32]:
using LinearAlgebra

In [43]:
A = [[140, 97, 74, 168, 131];;
[97, 106, 89, 131, 36];;
[74, 89, 152, 144, 71];;
[168, 131, 144, 54, 142];;
[131, 36, 71, 142, 36]]

exerciseAEigenVals = eigen(A)
A_eigv = eigvals(A)
testVal = exerciseAEigenVals.values
println(A_eigv == testVal)
println(A_eigv)

false
[-128.4932276480215, -55.887784553056875, 42.75216727931892, 87.16111477514524, 542.4677301466141]


In [44]:
@assert A_eigv ≈ [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

# Changed assert to approximation so that the eigenvalues are properly calculated to enough significant digits.


#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [48]:
A_diag = zeros(Float64, 5, 5)
for i in 1:5
    A_diag[i, i] = Diagonal(A_eigv)[i, i]
    A_diag[i, i] = round(A_diag[i, i], sigdigits=6)
end
A_diag


5×5 Matrix{Float64}:
 -128.493    0.0      0.0      0.0       0.0
    0.0    -55.8878   0.0      0.0       0.0
    0.0      0.0     42.7522   0.0       0.0
    0.0      0.0      0.0     87.1611    0.0
    0.0      0.0      0.0      0.0     542.468

In [49]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]
    

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [ ]:
A_lowertri = zeros(Float64, 5, 5)
for i in 1:5
    for j in 1:5
        if i >= j
            A_lowertri[i, j] = A[i,j]
        end
    end
end
A_lowertri

5×5 Matrix{Float64}:
 140.0    0.0    0.0    0.0   0.0
  97.0  106.0    0.0    0.0   0.0
  74.0   89.0  152.0    0.0   0.0
 168.0  131.0  144.0   54.0   0.0
 131.0   36.0   71.0  142.0  36.0

In [ ]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]